# **Importing the necessary libraries**
---


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Loading our dataset.

In [0]:
df_nor = pd.read_csv('my_work/HousePRice/House_Price_Details (1).csv')

In [0]:
df_cat = pd.read_csv('my_work/HousePRice/House_Price_Data_with_cat_2.csv')

In [6]:
df_nor.shape,df_cat.shape

((416960, 17), (416960, 92))

In [7]:
df_nor.describe()

,Mission_Nb,Sold_Price,Customer_Nb,Apt_Nb,Community_Loan,Nb_of_Rooms,Street_Nb,Area
count,4.169600e+05,4.169600e+05,416960.000000,416960.000000,4.169600e+05,416960.000000,416960.000000,416960.000000
mean,2.726752e+08,1.128368e+06,282.643865,279.374585,2.738553e+05,2.898705,31.434241,67.606634
std,3.662127e+08,9.735070e+05,730.335167,865.255771,2.473956e+05,0.977302,39.647222,28.307834
min,1.000000e+00,0.000000e+00,1.000000,1.000000,-6.910390e+03,0.000000,1.000000,0.000000
25%,1.000027e+08,3.600000e+05,34.000000,15.000000,1.075035e+05,2.000000,7.000000,53.000000
50%,1.000154e+08,8.900000e+05,89.000000,44.000000,2.012346e+05,3.000000,18.000000,68.000000
75%,1.189029e+08,1.650000e+06,286.000000,135.000000,3.625535e+05,4.000000,39.000000,86.000000
max,1.000099e+09,7.450000e+06,8178.000000,82101.000000,1.922807e+06,8.000000,284.000000,256.000000


In [8]:
df_nor.Sold_Price.max(),df_nor.Sold_Price.min()

(7450000.0, 0.0)

# Dataset is still having some invalid data in Sold_Price and Community_Loan column.

In [0]:
df_nor.drop(df_nor[df_nor.Sold_Price <= 10000].index, axis = 0, inplace = True)

In [10]:
df_nor.Sold_Price.max(),df_nor.Sold_Price.min()

(7450000.0, 10005.0)

In [0]:
df_nor.drop(df_nor[df_nor.Community_Loan <= 1000].index,axis = 0,inplace = True)

In [12]:
df_nor.shape

(386920, 17)

In [0]:
df_cat.drop(df_cat[df_cat.Sold_Price <= 10000].index, axis = 0, inplace = True)

In [14]:
df_cat.Sold_Price.max(),df_cat.Sold_Price.min()

(7450000.0, 10005.0)

In [15]:
df_cat.Community_Loan.max(),df_cat.Community_Loan.min()

(1922807.11, -6910.39)

In [0]:
df_cat.Community_Loan = abs(df_cat.Community_Loan)

In [0]:
df_cat.drop(df_cat[df_cat.Community_Loan <= 1000].index,axis = 0,inplace = True)

In [18]:
df_cat.shape

(386921, 92)


# **Random Forest**
---
---



# Label Encodeing categorical features for Random Forest Algorithm.

In [0]:
from sklearn.preprocessing import LabelEncoder
df_nor.Post_Code = df_nor.Post_Code.astype('str')
X_enc = df_nor.apply(LabelEncoder().fit_transform)

In [0]:
X_nf = X_enc.drop('Sold_Price',axis = 1)
y_nf = X_enc['Sold_Price']

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
xx_train, xx_test, yy_train, yy_test = train_test_split(X_nf, y_nf, test_size=0.10, random_state=5)

In [69]:
regr = RandomForestRegressor(bootstrap = True,max_depth = 200,max_features = 5,min_samples_leaf = 10,min_samples_split = 15,n_estimators = 3000)
regr.fit(xx_train, yy_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=200, max_features=5, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=10,
                      min_samples_split=15, min_weight_fraction_leaf=0.0,
                      n_estimators=3000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
p = regr.predict(xx_test)

In [71]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(yy_test, p))
print('Mean Squared Error:', metrics.mean_squared_error(yy_test, p))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(yy_test, p)))

Mean Absolute Error: 966.0749941612612
Mean Squared Error: 2346581.012862974
Root Mean Squared Error: 1531.8554151299572


In [0]:
import copy
from sklearn.feature_extraction import FeatureHasher
X_train_hash = copy.copy(xx_train)
X_test_hash = copy.copy(xx_test)
for i in range(X_train_hash.shape[1]):
    X_train_hash.iloc[:,i]=X_train_hash.iloc[:,i].astype('str')
for i in range(X_test_hash.shape[1]):
    X_test_hash.iloc[:,i]=X_test_hash.iloc[:,i].astype('str')
h = FeatureHasher(n_features=100,input_type="string")
X_train_hash = h.transform(X_train_hash.values)
X_test_hash = h.transform(X_test_hash.values)
#l.fit(X_train_hash,y_train)
#y_pred = l.predict_proba(X_test_hash)
#print(log_loss(y_test,y_pred))#0.4

In [0]:
y_pred = regr.predict(X_test_hash)

In [0]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(yy_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(yy_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(yy_test, y_pred)))

Mean Absolute Error: 2182.985625990921
Mean Squared Error: 7045053.604778543
Root Mean Squared Error: 2654.2519859234435


# **Let's Apply Neural Network of Dataset to predict the house prices.**
---
---

In [0]:
X = df_cat.drop('Sold_Price',axis = 1)

In [0]:
y = np.array(df_cat['Sold_Price'])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [0]:
# Scale the data to be between -1 and 1
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [71]:
X_train.shape

(348228, 91)

In [72]:
X_test.shape

(38693, 91)

In [73]:
y_train.shape

(348228,)

In [74]:
y_test.shape

(38693,)

In [75]:
%tensorflow_version 2.x
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from xgboost import XGBRegressor
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

In [0]:
from keras import initializers
NN_model = tf.keras.models.Sequential()

# batch Normalization 
NN_model.add(tf.keras.layers.BatchNormalization())

# The Input Layer :
NN_model.add(tf.keras.layers.Dense(10,input_dim = X_train.shape[1], activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=None)))

# batch Normalization 
NN_model.add(tf.keras.layers.BatchNormalization())

# The Hidden Layers :
NN_model.add(tf.keras.layers.Dense(100,activation='relu',))
NN_model.add(tf.keras.layers.Dense(50,activation='relu'))
NN_model.add(tf.keras.layers.Dense(25,activation='relu'))

# The Output Layer :
NN_model.add(tf.keras.layers.Dense(1, kernel_initializer='normal',activation='linear'))

In [0]:
# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = False, mode ='auto')
callbacks_list = [checkpoint]

In [81]:
NN_model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Train on 278582 samples, validate on 69646 samples
Epoch 1/15
278560/278582 [============================>.] - ETA: 0s - loss: 663499.3079 - mean_absolute_error: 663500.0625
Epoch 00001: saving model to Weights-001--710134.13191.hdf5
278582/278582 [==============================] - 38s 135us/sample - loss: 663507.1258 - mean_absolute_error: 663507.8750 - val_loss: 710134.1319 - val_mean_absolute_error: 710133.9375
Epoch 2/15
278400/278582 [============================>.] - ETA: 0s - loss: 625929.2220 - mean_absolute_error: 625928.2500
Epoch 00002: saving model to Weights-002--806695.00284.hdf5
278582/278582 [==============================] - 37s 132us/sample - loss: 625920.0728 - mean_absolute_error: 625919.0625 - val_loss: 806695.0028 - val_mean_absolute_error: 806694.0625
Epoch 3/15
278560/278582 [============================>.] - ETA: 0s - loss: 619485.6581 - mean_absolute_error: 619484.6875
Epoch 00003: saving model to Weights-003--686269.08577.hdf5
278582/278582 [=================

In [0]:
# Load wights file of the best model :
wights_file = 'Weights-015--709123.68326.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [0]:
#def make_submission(prediction, sub_name):
#  my_submission = pd.DataFrame({'Y_test':y_test,'SalePrice':prediction})
#   my_submission.to_csv('{}.csv'.format(sub_name),index=False)
#  print('A submission file has been made')

#predictions = NN_model.predict(X_test)
#make_submission(predictions[:,0],'submission(NN)_1.csv')

In [0]:
y_pred_1 = NN_model.predict(X_test)

In [92]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_1))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_1)))

Mean Absolute Error: 718700.66825545
Mean Squared Error: 16537722717904.006
Root Mean Squared Error: 4066659.897004421


# **Support Vector Regressor**
---
---


In [0]:
from sklearn.svm import SVR

In [0]:
X_sample = X_enc.sample(frac=0.1, replace=True, random_state=1)

In [35]:
x_svr.shape

(38692, 17)

In [0]:
x_sv = x_svr.drop('Sold_Price',axis = 1)

In [0]:
y_sv = x_svr['Sold_Price']

In [0]:
clf = SVR(C=1.0, epsilon=0.2)

In [0]:
Xsvr_train, Xsvr_test, ysvr_train, ysvr_test = train_test_split(x_sv, y_sv, test_size=0.10, random_state=42)

In [40]:
clf.fit(Xsvr_train,ysvr_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
pred_svr = clf.predict(Xsvr_test)

In [48]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(ysvr_test, pred_svr))
print('Mean Squared Error:', metrics.mean_squared_error(ysvr_test, pred_svr))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ysvr_test, pred_svr)))
print('R^2 value:', clf.score(Xsvr_test,ysvr_test))

Mean Absolute Error: 2471.431239328415
Mean Squared Error: 11355892.434591908
Root Mean Squared Error: 3369.8505062675863
R^2 value: -0.147854130989318


In [0]:
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt

In [0]:
svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1),
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})

# **Bagging Regressor**
***
---

In [0]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor

In [0]:
X_bag = X_sample.drop('Sold_Price',axis = 1)

In [0]:
y_bag = X_sample['Sold_Price']

In [0]:
Xbag_train, Xbag_test, ybag_train, ybag_test = train_test_split(X_bag, y_bag, test_size=0.10, random_state=42)

In [0]:
bag = BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=0)

In [62]:
bag.fit(Xbag_train, ybag_train)

BaggingRegressor(base_estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                    epsilon=0.1, gamma='scale', kernel='rbf',
                                    max_iter=-1, shrinking=True, tol=0.001,
                                    verbose=False),
                 bootstrap=True, bootstrap_features=False, max_features=1.0,
                 max_samples=1.0, n_estimators=10, n_jobs=None, oob_score=False,
                 random_state=0, verbose=0, warm_start=False)

In [0]:
pred_bag = bag.predict(Xbag_test)

In [64]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(ybag_test, pred_bag))
print('Mean Squared Error:', metrics.mean_squared_error(ybag_test, pred_bag))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ybag_test, pred_bag)))
print('R^2 value:', bag.score(Xsvr_test,ybag_test))

Mean Absolute Error: 2471.2178904461625
Mean Squared Error: 11331518.893768089
Root Mean Squared Error: 3366.232150902265
R^2 value: -0.14539045235880943
